# Introduction

This notebook takes the monthly structured warrants report on Bursa Malaysia and applies some data wranggling to it:

See the [bm_report_converter](https://github.com/hsm207/bm_report_converter) repo for details.

# Libraries

In [1]:
import pandas as pd
from bfinance import Equity
import time
import numpy as np
from tqdm import tqdm

# Setup

In [2]:
%cd ..

/workspaces/bm_sector_rotation


Path to the monthly report:

In [3]:
DATA_FILE = "./data/2021-02_bm_monthly_sw.csv"

Path to save the results:

In [4]:
OUT_FILE = "./data/2021-02_bm_monthly_sw_processed.parquet"

To enable tqdm with pandas:

In [5]:
tqdm.pandas()

# Data

In [6]:
column_names = ["No.", 
                "Stock Code", 
                "Stock Name", 
                "Security Type", 
                "No. of Warrants Outstanding", 
                "Maturity Date", 
                "Exercise Price", 
                "Conversion Ratio", 
                "Share per Warrant", 
                "Premium (Discount)", 
                "Gearing",
                "dummy"]

df = pd.read_csv(DATA_FILE, 
                 header = 0,
                 names = column_names,
                 index_col = "Stock Code",
                 usecols = lambda x: x != 'dummy',
                 na_values = '-',
                 thousands = ',',
                 parse_dates = ["Maturity Date"],
                 infer_datetime_format = False,
                 date_parser = lambda x: pd.to_datetime(x, format = "%d/%m/%Y"),
                 dtype = {
                     'Gearing': 'float'
                 })

df.head(10)

,No.,Stock Name,Security Type,No. of Warrants Outstanding,Maturity Date,Exercise Price,Conversion Ratio,Share per Warrant,Premium (Discount),Gearing
Stock Code,,,,,,,,,,
070364,1,A50CHIN-C64,Call WARRANTS,29000000,2021-03-31,HKD 20.000,1.5 for 1,0.67,NaN,NaN
070366,2,A50CHIN-C66,Call WARRANTS,24000000,2021-03-31,HKD 18.500,1.5 for 1,0.67,NaN,NaN
070368,3,A50CHIN-C68,Call WARRANTS,26000000,2021-06-30,HKD 21.500,1.5 for 1,0.67,NaN,NaN
070370,4,A50CHIN-C70,Call WARRANTS,28000000,2021-06-30,HKD 19.500,1.5 for 1,0.67,NaN,NaN
070372,5,A50CHIN-C72,Call WARRANTS,26000000,2021-09-30,HKD 28.000,2 for 1,0.50,NaN,NaN
070374,6,A50CHIN-C74,Call WARRANTS,30000000,2021-09-30,HKD 25.000,2 for 1,0.50,NaN,NaN
070349,7,A50CHIN-H49 #,PUT WARRANTS,32000000,2021-03-31,HKD 18.000,1.5 for 1,0.67,NaN,NaN
070351,8,A50CHIN-H51 #,PUT WARRANTS,30000000,2021-03-31,HKD 14.500,1.5 for 1,0.67,NaN,NaN
070353,9,A50CHIN-H53 #,PUT WARRANTS,33000000,2021-06-30,HKD 17.000,1.5 for 1,0.67,NaN,NaN


# Sanity Checks

Check that conversion from pdf has extracted all the warrants by looking at the sequence of the `No.` column:

In [7]:
assert df["No."].diff().dropna().sum() == df.shape[0] - 1

# Feature Engineering

Add some useful features for later analysis.

## Days to Expiry

Calculate how many days to expiry and ignore warrants with less than 60 days to expiry:

In [8]:
current_date = pd.Timestamp(pd.Timestamp.now().date())

df["time_to_expiry"] = df["Maturity Date"] - current_date
df = df.query('time_to_expiry > @pd.Timedelta(60, "day")')

num_warrants, _ = df.shape
print(f"There are {num_warrants:d} warrants with more than 60 days to expiry")

There are 584 warrants with more than 60 days to expiry


## Warrant Type

Simplify warrant type i.e. call or put:

In [9]:
df.loc[:, "Security Type"] = df["Security Type"].str.split(' ').str[0].str.lower()

In [10]:
df.sample(n = 10)

,No.,Stock Name,Security Type,No. of Warrants Outstanding,Maturity Date,Exercise Price,Conversion Ratio,Share per Warrant,Premium (Discount),Gearing,time_to_expiry
Stock Code,,,,,,,,,,,
5292CE,927,UWC-CE,call,35000000,2021-09-06,2.750,15 for 1,0.07,65.89%,0.81,162 days
71131C,874,TOPGLOV-C1C,call,35000000,2021-11-08,7.880,25 for 1,0.04,79.01%,3.49,225 days
181839,85,BURSA-C39,call,100000000,2021-05-31,13.280,20 for 1,0.05,48.13%,45.50,64 days
521631,196,DSONIC-C31,call,50000000,2021-07-29,0.650,2.5 for 1,0.40,40.78%,6.87,123 days
534783,846,TENAGA-C83,call,50000000,2021-07-22,10.300,20 for 1,0.05,12.87%,9.18,116 days
102373,107,CIMB-C73,call,50000000,2021-07-29,3.200,5.5 for 1,0.18,1.85%,3.58,123 days
7148CQ,174,DPHARMA-CQ,call,60000000,2021-10-27,3.880,10 for 1,0.10,47.74%,4.43,213 days
165167,604,MRCB-C67,call,50000000,2021-06-28,0.530,2 for 1,0.50,37.35%,10.38,92 days
7153HC,518,KOSSAN-HC #,put,100000000,2021-05-31,5.500,50 for 1,0.02,81.01%,0.83,64 days


## Warrant Profile

Get the profile of each warrant:

In [11]:
import traceback

def random_sleep():
    secs = np.random.randint(2, 10)
    time.sleep(secs)
    
def get_profile(stock_code):
    try:
        random_sleep()
        return Equity(stock_code).instrument_profile
    except Exception as e:
        print(f"Failed to process {stock_code}!")
        traceback.print_exc()

In [12]:
%%time
warrant_profiles = df.index.to_series().progress_apply(get_profile).to_list()
warrant_profiles = pd.DataFrame(warrant_profiles, index = df.index)

df = df.join(warrant_profiles)
df.sample(n = 10)

100%|██████████| 584/584 [1:05:08<00:00,  6.69s/it]

CPU times: user 32.7 s, sys: 2.9 s, total: 35.6 s
Wall time: 1h 5min 8s


,No.,Stock Name,Security Type,No. of Warrants Outstanding,Maturity Date,Exercise Price,Conversion Ratio,Share per Warrant,Premium (Discount),Gearing,time_to_expiry,underlying,issuer,market,sector
Stock Code,,,,,,,,,,,,,,,
4731CK,733,SCIENTX-CK,call,40000000,2021-08-26,3.600,15 for 1,0.07,43.66%,1.85,151 days,SCIENTEX BERHAD,RHB,Structured Warrants,INDUSTRIAL PRODUCTS & SERVICES
7090CA,19,AHEALTH-CA,call,100000000,2021-05-31,3.880,12 for 1,0.08,43.38%,4.17,64 days,APEX HEALTHCARE BERHAD,MIBB,Structured Warrants,HEALTH CARE
3069CO,577,MFCB-CO,call,40000000,2021-06-28,8.000,10 for 1,0.10,13.78%,10.25,92 days,MEGA FIRST CORPORATION BERHAD,RHB,Structured Warrants,UTILITIES
715327,515,KOSSAN-C27,call,60000000,2021-10-27,5.000,12 for 1,0.08,72.15%,2.19,213 days,KOSSAN RUBBER INDUSTRIES BERHAD,RHB,Structured Warrants,HEALTH CARE
102369,103,CIMB-C69,call,100000000,2021-08-30,3.280,4 for 1,0.25,1.15%,3.94,155 days,CIMB GROUP HOLDINGS BERHAD,MIBB,Structured Warrants,FINANCIAL SERVICES
129558,657,PBBANK-C58,call,35000000,2021-11-08,4.000,18 for 1,0.06,89.18%,1.07,225 days,PUBLIC BANK BERHAD,KIBB,Structured Warrants,FINANCIAL SERVICES
527918,741,SERBADK-C18,call,150000000,2021-05-31,1.550,2 for 1,0.50,5.26%,6.84,64 days,SERBA DINAMIK HOLDINGS BERHAD,CIMB,Structured Warrants,ENERGY
106628,721,RHBBANKC28,call,21000000,2021-06-11,6.080,3.5 for 1,0.29,17.02%,20.65,75 days,RHB BANK BERHAD,MACQ,Structured Warrants,FINANCIAL SERVICES
524818,75,BAUTO-C18,call,150000000,2021-07-30,1.400,1.5 for 1,0.67,15.11%,9.40,124 days,BERMAZ AUTO BERHAD,CIMB,Structured Warrants,CONSUMER PRODUCTS & SERVICES


# Save Results 

Save the file:

In [13]:
df.reset_index().to_feather(OUT_FILE)

Re-read just to be sure:

In [14]:
pd.read_feather(OUT_FILE)\
    .set_index("Stock Code")\
    .sample(n=10)

,No.,Stock Name,Security Type,No. of Warrants Outstanding,Maturity Date,Exercise Price,Conversion Ratio,Share per Warrant,Premium (Discount),Gearing,time_to_expiry,underlying,issuer,market,sector
Stock Code,,,,,,,,,,,,,,,
5285CS,756,SIMEPLT-CS,call,100000000,2021-07-30,5.380,5 for 1,0.20,17.45%,13.07,124 days,SIME DARBY PLANTATION BERHAD,MIBB,Structured Warrants,PLANTATION
674215,970,YTLPOWR-C15,call,60000000,2021-08-27,0.750,1.5 for 1,0.67,14.00%,7.14,152 days,YTL POWER INTERNATIONAL BHD,RHB,Structured Warrants,UTILITIES
129558,657,PBBANK-C58,call,35000000,2021-11-08,4.000,18 for 1,0.06,89.18%,1.07,225 days,PUBLIC BANK BERHAD,KIBB,Structured Warrants,FINANCIAL SERVICES
115570,556,MAYBANKC70,call,14000000,2021-06-30,8.880,5.5 for 1,0.18,15.72%,19.47,94 days,MALAYAN BANKING BERHAD,MACQ,Structured Warrants,FINANCIAL SERVICES
5102CX,294,GCB-CX,call,19000000,2021-07-16,3.480,6 for 1,0.17,38.72%,12.67,110 days,GUAN CHONG BERHAD,MACQ,Structured Warrants,CONSUMER PRODUCTS & SERVICES
0128CR,274,FRONTKN-CR,call,35000000,2021-07-26,4.200,6 for 1,0.17,10.56%,3.34,120 days,FRONTKEN CORPORATION BERHAD,KIBB,Structured Warrants,TECHNOLOGY
06509T,228,FBMKLCI-C9T,call,100000000,2021-07-30,1588.000,1000 for 1,0.00,None,NaN,124 days,FTSE BURSA MALAYSIA KLCI INDEX,MIBB,Structured Warrants,STRUCTURED WARRANTS
539891,287,GAMUDA-C91,call,150000000,2021-07-30,3.900,4 for 1,0.25,21.32%,23.79,124 days,GAMUDA BERHAD,CIMB,Structured Warrants,CONSTRUCTION
539890,286,GAMUDA-C90,call,14000000,2021-06-11,4.080,3 for 1,0.33,27.93%,18.50,75 days,GAMUDA BERHAD,MACQ,Structured Warrants,CONSTRUCTION
